<a href="https://colab.research.google.com/github/joonasrooben/OMML_JAB/blob/main/OMML_projet_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#OMML PROJECT

### Imports

In [13]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import math
import scipy as sp
from scipy.optimize import minimize

### Data analysis

In [14]:
data = pd.read_csv('DATA (1).csv')

In [15]:
data.head(5)

,x1,x2,y
0,1.725110,2.665940,0.101883
1,-1.137973,-1.059821,2.172760
2,1.652338,2.958089,0.118859
3,1.866858,-1.740966,-0.289316
4,-1.760604,1.350072,0.104652


In [16]:
len(data)

250

### Split the dataset

We put 0.255 for the test_size instead of 0.25 to be sure to have 186 observations in the training_data

In [17]:
X_train, X_test, y_train, y_test = train_test_split(data[['x1','x2']], data[['y']], test_size=0.255, random_state=1990243)

In [18]:
n = len(X_train)
n

186

In [19]:
X_train

,x1,x2
236,0.890055,-2.955518
134,0.355821,-2.385734
129,1.795969,0.914778
209,0.070684,-2.177204
52,1.674262,0.182846
...,...,...
244,1.557484,-0.167087
158,-1.730929,-0.657224
33,-1.367034,2.838806
35,-1.733677,-1.672311


### Question 1


#### Question 1.1


**Activation function**

We define the acivation function.

In [20]:
def activationFunction(t, sigma = 1):
  return (np.exp(2*sigma*t)-1)/(np.exp(2*sigma*t)+1)

**Weight Initialization for Tanh**

The current standard approach for initialization of the weights of neural network layers and nodes that use the Sigmoid or TanH activation function is called “glorot” or “xavier” initialization

The xavier initialization method is calculated as a random number with a uniform probability distribution (U) between the range -(1/sqrt(n)) and 1/sqrt(n), where n is the number of inputs to the node.

weight = U [-(1/sqrt(n)), 1/sqrt(n)]

Let's define all the hyperparameters here
Let's put 3 neurons in our hidden layer

In [21]:
N = 5
rho = 1
sigma = 1

In [22]:
inputs = X_train
inputs = inputs.to_numpy()
inputs.shape

(186, 2)

In [23]:
def createWeightsForOneNeuron(N):
  initial_weights_one_neuron = []
  for i in range(0,N):
    initial_weights_one_neuron.append(np.random.uniform(low = -(1/math.sqrt(N)), high = 1/math.sqrt(N)))
  return initial_weights_one_neuron

def create_initial_weights(N):
  initial_weights = []
  for i in range(0,2):
    initial_weights.append(createWeightsForOneNeuron(N))
  return initial_weights

weightsW = np.asarray(create_initial_weights(N))


In [24]:
mult = np.matmul(inputs, weightsW)

Create vector of bias


In [25]:
bias = np.ones((N,1))

Creation of the linear function

∑wjixi+bj


In [26]:
linearResult = mult + bias.T
linearResult.shape

(186, 5)

Now the step is the get this number into the activation function

In [27]:
acti = activationFunction(linearResult)
acti.shape

(186, 5)

We generate weigths for hidden layers


In [28]:
def createWeightsForHiddenLayer(N):
  weightHiddenLayer = []
  for i in range(0,N):
    weightHiddenLayer.append(np.random.uniform(low = -(1/math.sqrt(N)), high = 1/math.sqrt(N)))
  return weightHiddenLayer

In [29]:
v = createWeightsForHiddenLayer(N)
v = np.array(v)

In [30]:
f_x = np.matmul(v,acti.T)
f_x.shape


(186,)

Now we have the MLP. We'll computed the regularized training error function E(v,w,b)

In [31]:
y = y_train
y = y.to_numpy()
y = y.reshape((-1))
y.shape

(186,)

In [32]:
P = len(X_train)

In [33]:
firstTermE = (1/2*P)* sum((f_x - y)**2)

First create the regularization term



In [34]:
omega = [v, weightsW, bias]

In [35]:

def computeRegularizationTerm(omega):
    normV = np.linalg.norm(omega[0]**2)
    normW = np.linalg.norm(omega[1]**2)
    normB = np.linalg.norm(omega[2]**2)
    return normV + normW + normB

omega_norm = computeRegularizationTerm(omega)

regularizationTerm = (rho/2)*(omega_norm)

In [36]:
E = firstTermE + regularizationTerm
E

41298.12990724974

To resume a big function we have 

In [56]:
def expectationFunction(omega ,input, target, N, rho, sigma):

  W = omega[:-2*N].reshape((2,N))
  v = omega[-N:]
  bias = omega[-2*N:-N]


  x = input.to_numpy()

  mult = np.matmul(inputs, W)

  linearResult = mult + bias.T

  acti = activationFunction(linearResult, sigma)

  f_x = np.matmul(v,acti.T)

  y = target.to_numpy()
  y = y.reshape((-1)) 

  P = len(input)

  firstTermE = (1/2*P)* sum((f_x - y)**2)

  omega = [v, W, bias]

  regularizationTerm = (rho/2)*(omega_norm)

  E = firstTermE + regularizationTerm

  return  E

In [73]:
N = 20
rho = 1e-5
sigma = 1
W = np.array(create_initial_weights(N))
W.size
v = np.array(createWeightsForHiddenLayer(N))
bias = np.ones((N,1))
omega = np.append(np.append(W.flatten(), bias), v)
W.shape


(2, 20)

In [70]:
expectationFunction(omega, X_train ,y_train ,N ,rho ,sigma)

22968.60476357614

In [74]:
res = minimize(expectationFunction, omega, args=((X_train ,y_train ,N ,rho ,sigma)), method = 'CG', options={"maxiter":1000})

In [75]:
print(res)

     fun: 51.83377847694325
     jac: array([ 7.08912292e+01,  9.32295671e+01, -4.51475482e+01,  3.06036329e+01,
        1.35249043e+00,  1.56418257e+01, -2.90938473e+00, -1.14573627e+01,
       -1.32503448e+01, -8.10931683e-01, -1.32562160e-01,  7.40621309e+01,
       -1.40413351e+01,  9.09232850e+01, -1.08164015e+01,  2.34243045e+01,
       -2.50804424e+00,  7.37625232e+01, -1.69900179e+00,  6.49709749e+00,
        8.34095435e+01, -3.07269239e+01,  6.15124178e+01,  4.47931428e+01,
       -3.58339806e+01,  1.43703569e+02, -3.34159546e+01, -4.93072839e+01,
       -5.64144001e+01, -8.96390057e+00, -9.62756157e+00,  1.44299251e+02,
       -5.92705359e+01,  1.17933013e+02, -5.60406170e+01, -1.17443278e+02,
       -2.79059043e+01,  2.73282995e+01, -1.64017520e+01,  2.78879151e+01,
       -2.93906350e+01,  7.83554277e+01,  3.26276588e+01,  2.79694037e+01,
       -5.98010731e+00, -2.99209151e+01, -8.58070850e-01,  4.94844866e+00,
        5.77524281e+00, -2.77417898e+00, -1.02601604e+01, -1.3